# Regresión Logística

In [123]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


#### Leer los datos

In [124]:
# Leer los datos
df = pd.read_csv("train.csv")

# Selección de variables
variables_numericas = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'YearBuilt', 'KitchenAbvGr',
    'TotRmsAbvGrd', 'Fireplaces', 'SalePrice'
]
df = df[variables_numericas]

#### Clasificación de Precios

In [125]:
# Clasificación de precios
p40 = df['SalePrice'].quantile(0.40)
p90 = df['SalePrice'].quantile(0.90)

def clasificar_precio(precio):
    if precio <= p40:
        return "Económica"
    elif precio <= p90:
        return "Intermedia"
    else:
        return "Cara"

#### Variables Dicotómicas

In [126]:
df['CategoriaPrecio'] = df['SalePrice'].apply(clasificar_precio)
df['Es_Cara'] = (df['CategoriaPrecio'] == 'Cara').astype(int)

#### Varables predictoras y respuesta

In [127]:
# Definir variables predictoras y respuesta
X = df.drop(columns=['SalePrice', 'CategoriaPrecio', 'Es_Cara'])
y = df['Es_Cara']

#### Separar en modelos de entrenamiento y prueba

In [128]:
# Separar en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, train_size=0.7, random_state=42, stratify=y
)

#### Modelo de Regresión Logística

In [129]:
# Modelo de regresión logística
modelo = LogisticRegression(solver='liblinear')
modelo.fit(X_train, y_train)

# Predicciones y Probabilidades de predicción
y_proba = modelo.predict_proba(X_test)[:, 1]

# Nuevo umbral de decisión
umbral = 0.35
y_pred = (y_proba >= umbral).astype(int)

#### Métricas y Resultados

In [130]:
# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cv = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')

# Resultados
print(f"Accuracy: {accuracy:.2f}")
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precisión promedio (validación cruzada): {cv.mean():.2f}")
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.95
Precisión: 0.75
Recall: 0.82
F1 Score: 0.78
Precisión promedio (validación cruzada): 0.95
Matriz de confusión:
[[382  12]
 [  8  36]]
